# Bin Problem

In [12]:
from docplex.cp.model import CpoModel

## Definding the Variables

In [24]:
items = [
    {"name": "A", "width": 3, "height": 3, "length": 10, "count": 10},
    {"name": "B", "width": 4, "height": 1, "length": 10, "count": 15},
    {"name": "C", "width": 2, "height": 5, "length": 10, "count": 10},
]

containers = [
    {"name": "S", "width": 20, "height": 50, "length": 100},
    {"name": "M", "width": 40, "height": 100, "length": 100},
]

## Initialize Model

In [25]:
model = CpoModel(name="3D_Bin_Packing")

## Variables

### Whether an item of type i is placed in container j

In [26]:
x = {
    (i, j): model.integer_var(0, items[i]["count"], name=f"x_{i}_{j}")
    for i in range(len(items))
    for j in range(len(containers))
}

In [27]:
y = {
    j: model.binary_var(name=f"y_{j}")
    for j in range(len(containers))
}

## Total number of containers used

In [28]:
num_containers_used = model.integer_var(0, len(containers), name="num_containers_used")

## Constraints

### 1. Ensure all items are packed

In [29]:
for i, item in enumerate(items):
    model.add(
        model.sum(x[i, j] for j in range(len(containers))) == item["count"]
    )

### 2. Ensure container capacities are respected

In [30]:
for j, container in enumerate(containers):
    model.add(
        model.sum(
            x[i, j] * items[i]["width"] * items[i]["height"] * items[i]["length"]
            for i in range(len(items))
        )
        <= container["width"] * container["height"] * container["length"]
    )

### 3. Ensure containers are only counted as used if they contain items

In [31]:
for j in range(len(containers)):
    model.add(
        model.sum(x[i, j] for i in range(len(items))) <= 100000 * y[j]  # Large constant
    )

## Objective Function

In [32]:
model.add(model.minimize(model.sum(y[j] for j in range(len(containers)))))

## Solving Model

In [33]:
solution = model.solve()

 ! --------------------------------------------------- CP Optimizer 22.1.1.0 --
 ! Minimization problem - 8 variables, 7 constraints
 ! Presolve      : 2 extractables eliminated
 ! Initial process time : 0.01s (0.01s extraction + 0.00s propagation)
 !  . Log search space  : 12.9 (before), 12.9 (after)
 !  . Memory usage      : 267.8 kB (before), 267.8 kB (after)
 ! Using parallel search with 16 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0          5                 -
 + New bound is 0
                        0          5    1   F        -
 + New bound is 1
 *             1        1  0.02s        1      (gap is 0.00%)
 ! ----------------------------------------------------------------------------
 ! Search completed, 1 solution found.
 ! Best objective         : 1 (optimal - effective tol. is 0)
 ! Best bound             : 1
 ! ---------------------------

## Displaying the Results

In [34]:
if solution:
    print("Optimal solution found!")
    for j, container in enumerate(containers):
        print(f"Container {container['name']}:")
        for i, item in enumerate(items):
            count = solution[x[i, j]]
            if count > 0:
                print(f"  Item {item['name']}: {count}")
    print(f"Total Containers Used: {sum(solution[y[j]] for j in range(len(containers)))}")
else:
    print("No solution found.")

Optimal solution found!
Container S:
  Item A: 10
  Item B: 15
  Item C: 10
Container M:
Total Containers Used: 1


# End of The Code